# DataFrames Basics Exercises

## Prerrequisites

Install Spark and Java in VM

In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [2]:
ls -l # check the .tgz is there

total 267680
drwxr-xr-x 1 root root      4096 Dec  7 14:41 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [3]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [4]:
!pip install -q findspark

Defining the environment

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [6]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFramesBasics Exercises") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [7]:
spark

In [8]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [9]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [10]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/movies.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/cars.json -P /dataset

## DataFrames Basics Exercises

1) Create a manual DF describing smartphones
  - maker
  - model
  - screen dimension
  - camera megapixels
  
2) Read another file from the dataset/ folder, e.g. movies.json
  - print its schema
  - count the number of rows, call count()

In [36]:
import pandas as pd

#Datos del DF
data = [["samung", "1","12","12"],["iphone", "1","12","12"], ["xiaomi", "1","12","12"]]

#DF sin usar pandas
df1 = spark.createDataFrame(data, schema="maker STRING NOT NULL, model STRING, screen_dimension STRING, camera_megapixels STRING")

#DF usando pandas
pdf = pd.DataFrame(data, columns=["maker", "model", "screen_dimension", "camera_megapixels"])
df2 = spark.createDataFrame(pdf)


In [26]:
df1.show()

+------+-----+----------------+-----------------+
| maker|model|screen_dimension|camera_megapixels|
+------+-----+----------------+-----------------+
|samung|    1|              12|               12|
|iphone|    1|              12|               12|
|xiaomi|    1|              12|               12|
+------+-----+----------------+-----------------+



In [27]:
df2.show()

+------+-----+----------------+-----------------+
| maker|model|screen_dimension|camera_megapixels|
+------+-----+----------------+-----------------+
|samung|    1|              12|               12|
|iphone|    1|              12|               12|
|xiaomi|    1|              12|               12|
+------+-----+----------------+-----------------+



In [31]:
df1.printSchema()

root
 |-- maker: string (nullable = false)
 |-- model: string (nullable = true)
 |-- screen_dimension: string (nullable = true)
 |-- camera_megapixels: string (nullable = true)



In [33]:
df2.printSchema()

root
 |-- maker NOT NULL: string (nullable = true)
 |-- model: string (nullable = true)
 |-- screen_dimension: string (nullable = true)
 |-- camera_megapixels: string (nullable = true)



In [42]:
movies_df = spark.read.json("../dataset/movies.json")
movies_df.printSchema()

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)



In [43]:
movies_df.count()

3201

## Columns and Expressions Exercises

1. Read the movies DF and select 2 columns of your choice
2. Create another column summing up the total profit of the movies = US_Gross + Worldwide_Gross + DVD sales. Are you pbtaining nulls? How you can solve it?
3. Select all COMEDY movies with IMDB rating above 6

Use as many versions as possible

# Read the movies DF and select 2 columns of your choice

In [41]:
movies_df = spark.read.json("../dataset/movies.json")
movies_df.show(2)

+-------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+------+--------------------+------------+--------+---------------+
|Creative_Type|Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+-------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+------+--------------------+------------+--------+---------------+
|         null|    null|   Gramercy|        6.1|      1071|          R|       null|          8000000|   12-Jun-98|                  null|            null|  null|      The Land Girls|        null|  146083|         146083|
|         null|    null|     Strand|        6.9|       207|          R|      Drama|           300000|    7-Aug-98|  

# Create another column summing up the total profit of the movies = US_Gross + Worldwide_Gross + DVD sales. Are you pbtaining nulls? How you can solve it?

In [50]:
movies_df = movies_df.fillna({"US_DVD_Sales": 0})
movies_df2 = movies_df.withColumn("Total_profit", col("US_Gross") + col("Worldwide_Gross") + col("US_DVD_Sales"))
movies_df2.show()

+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+------------+
|       Creative_Type|         Director|   Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Total_profit|
+--------------------+-----------------+--------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+------------+
|                null|             null|      Gramercy|        6.1|      1071|          R|       null|          8000000|   12-Jun-98|                  null|            null|               null|      The

# Select all COMEDY movies with IMDB rating above 6

In [62]:
subset_df = movies_df.where("IMDB_Rating > 6 and Major_Genre = 'Comedy'")
subset_df.show()

+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|       Creative_Type|         Director|         Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales| US_Gross|Worldwide_Gross|
+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|                null|             null|           Lionsgate|        6.8|       865|       null|     Comedy|           250000|   28-Aug-98|                  null|            null|                null|I Married a Stran

In [63]:
subset_df = movies_df.filter("IMDB_Rating > 6 and Major_Genre = 'Comedy'")
subset_df.show()

+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|       Creative_Type|         Director|         Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales| US_Gross|Worldwide_Gross|
+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|                null|             null|           Lionsgate|        6.8|       865|       null|     Comedy|           250000|   28-Aug-98|                  null|            null|                null|I Married a Stran

# La misma solución pero con una select

In [61]:
movies_df.createOrReplaceTempView("Movies")
query_df = spark.sql("SELECT * FROM Movies WHERE IMDB_Rating > 6 and Major_Genre = 'Comedy'")
query_df.show()

+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|       Creative_Type|         Director|         Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales| US_Gross|Worldwide_Gross|
+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|                null|             null|           Lionsgate|        6.8|       865|       null|     Comedy|           250000|   28-Aug-98|                  null|            null|                null|I Married a Stran

In [64]:
movies_df.registerTempTable("Movies_Table")
query_df = spark.sql("SELECT * FROM Movies_table WHERE IMDB_Rating > 6 and Major_Genre = 'Comedy'")
query_df.show()

/content/spark-3.3.1-bin-hadoop2/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|       Creative_Type|         Director|         Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|              Source|               Title|US_DVD_Sales| US_Gross|Worldwide_Gross|
+--------------------+-----------------+--------------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+--------------------+--------------------+------------+---------+---------------+
|                null|             null|           Lionsgate|        6.8|       865|       null|     Comedy|           250000|   28-Aug-98|                  null|            null|                null|I Married a Stran